<h1>ENB31 - Revisiting FFNs with K-fold cross validation</h1>
<h2> activation functions combinations </h2>

In [1]:
import tensorflow as tf
import keras
from keras import models
from keras import layers

import numpy as np
import os
import csv

import pickle
import time as tm

import matplotlib.pyplot as plt
import pre_process as pp

Using TensorFlow backend.


<h2> Loading the datasets</h2>

In [2]:
with open('train_data.txt', 'rb') as fp:
    train_data = pickle.load(fp)
    
with open('train_labels.txt', 'rb') as fp:
    train_labels = pickle.load(fp)
    
with open('test_data.txt', 'rb') as fp:
    test_data = pickle.load(fp)
    
with open('test_labels.txt', 'rb') as fp:
    test_labels = pickle.load(fp)

<h2>k-fold params</h2>

In [3]:
k = 4
num_val_samples = len(train_data)
print(num_val_samples)
val_split_size = num_val_samples/k
val_split_size = int(val_split_size)
print(val_split_size)
type(val_split_size)

72
18


int

<h2>Experiment1: a single layer</h2>

In [4]:
net1 = models.Sequential()
net1.add(layers.Dense((480), input_shape = (480,6)))
net1.add(layers.Flatten())
net1.add(layers.Dense(240,activation='relu'))
net1.add(layers.Dense(240,activation='relu'))
net1.add(layers.Dense(240,activation='softmax'))
net1.add(layers.Dense(7, activation = 'softmax'))
net1.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 480, 480)          3360      
_________________________________________________________________
flatten_1 (Flatten)          (None, 230400)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 240)               55296240  
_________________________________________________________________
dense_3 (Dense)              (None, 240)               57840     
_________________________________________________________________
dense_4 (Dense)              (None, 240)               57840     
_________________________________________________________________
dense_5 (Dense)              (None, 7)                 1687      
Total params: 55,416,967
Trainable para

In [5]:
net1.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
net1.save_weights('net1.h5')  # each split will start with same weights so can directly compare results

In [6]:

histories1 = []
results_list1 = []
index = 0
fold = val_split_size



start = tm.time()

for i in range (k):
    print(i)
    val_fold = train_data[index:fold]
    train_fold = np.delete(train_data, np.s_[index:fold],0)

    val_tgt = train_labels[index:fold]
    train_tgt = np.delete(train_labels, np.s_[index:fold])
    
    
    net1.load_weights('net1.h5')
    history = net1.fit(train_fold, train_tgt, epochs = 25, validation_data = (val_fold, val_tgt))

    histories1.append(history)
    
    
    results = net1.evaluate(test_data, test_labels)
    results_list1.append(results)
    print(results)
    
    index = index + val_split_size
    fold = fold + val_split_size
    
finish = tm.time()
duration1 = finish - start

0

Train on 54 samples, validate on 18 samples
Epoch 1/25
54/54 [==============================] - 2s 28ms/step - loss: 1.9446 - accuracy: 0.1852 - val_loss: 1.9291 - val_accuracy: 0.1667
Epoch 2/25
54/54 [==============================] - 1s 26ms/step - loss: 1.9296 - accuracy: 0.1667 - val_loss: 1.9273 - val_accuracy: 0.1111
Epoch 3/25
54/54 [==============================] - 1s 27ms/step - loss: 1.9309 - accuracy: 0.1852 - val_loss: 1.9194 - val_accuracy: 0.1111
Epoch 4/25
54/54 [==============================] - 1s 27ms/step - loss: 1.9306 - accuracy: 0.1852 - val_loss: 1.9401 - val_accuracy: 0.1111
Epoch 5/25
54/54 [==============================] - 1s 27ms/step - loss: 1.9336 - accuracy: 0.1852 - val_loss: 1.9394 - val_accuracy: 0.1111
Epoch 6/25
54/54 [==============================] - 1s 26ms/step - loss: 1.9332 - accuracy: 0.1852 - val_loss: 1.9386 - val_accuracy: 0.1111
Epoch 7/25
54/54 [==============================] - 1s 26ms/step - loss: 1.9326 - accuracy: 0.1852 - val_lo

Epoch 5/25
54/54 [==============================] - 1s 26ms/step - loss: 1.9175 - accuracy: 0.2037 - val_loss: 1.9793 - val_accuracy: 0.1111
Epoch 6/25
54/54 [==============================] - 1s 27ms/step - loss: 1.9160 - accuracy: 0.2037 - val_loss: 1.9798 - val_accuracy: 0.1111
Epoch 7/25
54/54 [==============================] - 1s 26ms/step - loss: 1.9142 - accuracy: 0.2037 - val_loss: 1.9802 - val_accuracy: 0.1111
Epoch 8/25
54/54 [==============================] - 1s 27ms/step - loss: 1.9127 - accuracy: 0.2037 - val_loss: 1.9807 - val_accuracy: 0.1111
Epoch 9/25
54/54 [==============================] - 1s 27ms/step - loss: 1.9108 - accuracy: 0.2037 - val_loss: 1.9812 - val_accuracy: 0.1111
Epoch 10/25
54/54 [==============================] - 1s 27ms/step - loss: 1.9095 - accuracy: 0.2037 - val_loss: 1.9817 - val_accuracy: 0.1111
Epoch 11/25
54/54 [==============================] - 1s 26ms/step - loss: 1.9077 - accuracy: 0.2037 - val_loss: 1.9823 - val_accuracy: 0.1111
Epoch 12/25

In [7]:
avg = np.mean(results_list1, axis=0)
std_dev = np.std(results_list1, axis=0)
print('Average loss and accuracy scores were: ', avg)
print('Standard deviation for loss & accuracy: ', std_dev)
print('Duration in seconds: ', int(duration1))

Average loss and accuracy scores were:  [1.91898298 0.18333334]
Standard deviation for loss & accuracy:  [0.0089379  0.02886751]
Duration in seconds:  145


<h2> Expriment2:</h2>

In [8]:
net2 = models.Sequential()
net2.add(layers.Dense((480), input_shape = (480,6)))
net2.add(layers.Flatten())
net2.add(layers.Dense(240,activation='relu'))
net2.add(layers.Dense(240,activation='relu'))
net2.add(layers.Dense(240,activation='sigmoid'))
net2.add(layers.Dense(7, activation = 'softmax'))
net2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 480, 480)          3360      
_________________________________________________________________
flatten_2 (Flatten)          (None, 230400)            0         
_________________________________________________________________
dense_7 (Dense)              (None, 240)               55296240  
_________________________________________________________________
dense_8 (Dense)              (None, 240)               57840     
_________________________________________________________________
dense_9 (Dense)              (None, 240)               57840     
_________________________________________________________________
dense_10 (Dense)             (None, 7)                 1687      
Total params: 55,416,967
Trainable params: 55,416,967
Non-trainable params: 0
__________________________________________

In [9]:
net2.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
net2.save_weights('net2.h5')  # each split will start with same weights so can directly compare results

histories2 = []
results_list2 = []
index = 0
fold = val_split_size



start = tm.time()

for i in range (k):
    print(i)
    val_fold = train_data[index:fold]
    train_fold = np.delete(train_data, np.s_[index:fold],0)

    val_tgt = train_labels[index:fold]
    train_tgt = np.delete(train_labels, np.s_[index:fold])
    
    
    net2.load_weights('net2.h5')
    history = net2.fit(train_fold, train_tgt, epochs = 25, validation_data = (val_fold, val_tgt))

    histories2.append(history)
    
    
    results = net2.evaluate(test_data, test_labels)
    results_list2.append(results)
    print(results)
    
    index = index + val_split_size
    fold = fold + val_split_size

finish = tm.time()
duration2 = finish - start

0
Train on 54 samples, validate on 18 samples
Epoch 1/25
54/54 [==============================] - 2s 29ms/step - loss: 2.3388 - accuracy: 0.1111 - val_loss: 2.3744 - val_accuracy: 0.1667
Epoch 2/25
54/54 [==============================] - 1s 26ms/step - loss: 2.0631 - accuracy: 0.1852 - val_loss: 1.7409 - val_accuracy: 0.2778
Epoch 3/25
54/54 [==============================] - 1s 27ms/step - loss: 1.8952 - accuracy: 0.1481 - val_loss: 1.6905 - val_accuracy: 0.2778
Epoch 4/25
54/54 [==============================] - 1s 28ms/step - loss: 1.8239 - accuracy: 0.1481 - val_loss: 1.6178 - val_accuracy: 0.3889
Epoch 5/25
54/54 [==============================] - 1s 27ms/step - loss: 1.6991 - accuracy: 0.3519 - val_loss: 1.5169 - val_accuracy: 0.3889
Epoch 6/25
54/54 [==============================] - 1s 26ms/step - loss: 1.6009 - accuracy: 0.3333 - val_loss: 1.4388 - val_accuracy: 0.3889
Epoch 7/25
54/54 [==============================] - 1s 27ms/step - loss: 1.5163 - accuracy: 0.3519 - val_los

54/54 [==============================] - 1s 26ms/step - loss: 1.6584 - accuracy: 0.4630 - val_loss: 2.1005 - val_accuracy: 0.2778
Epoch 7/25
54/54 [==============================] - 1s 26ms/step - loss: 1.6514 - accuracy: 0.4630 - val_loss: 1.8621 - val_accuracy: 0.3333
Epoch 8/25
54/54 [==============================] - 1s 26ms/step - loss: 1.5857 - accuracy: 0.4259 - val_loss: 1.8887 - val_accuracy: 0.2778
Epoch 9/25
54/54 [==============================] - 1s 27ms/step - loss: 1.5291 - accuracy: 0.5370 - val_loss: 1.7292 - val_accuracy: 0.3333
Epoch 10/25
54/54 [==============================] - 1s 26ms/step - loss: 1.4229 - accuracy: 0.4815 - val_loss: 1.6140 - val_accuracy: 0.3333
Epoch 11/25
54/54 [==============================] - 1s 27ms/step - loss: 1.3450 - accuracy: 0.5370 - val_loss: 1.6124 - val_accuracy: 0.3333
Epoch 12/25
54/54 [==============================] - 1s 27ms/step - loss: 1.3036 - accuracy: 0.5926 - val_loss: 1.6196 - val_accuracy: 0.2222
Epoch 13/25
54/54 [==

In [12]:
avg = np.mean(results_list2, axis=0)
std_dev = np.std(results_list2, axis=0)
print('Average loss and accuracy scores were: ', avg)
print('Standard deviation for loss & accuracy: ', std_dev)
print('Duration in seconds: ', int(duration2))

Average loss and accuracy scores were:  [0.55764947 0.75000001]
Standard deviation for loss & accuracy:  [0.07860766 0.02886751]
Duration in seconds:  202


In [11]:
net2.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
net2.save_weights('net2.h5')  # each split will start with same weights so can directly compare results

histories2 = []
results_list2 = []
index = 0
fold = val_split_size



start = tm.time()

for i in range (k):
    print(i)
    val_fold = train_data[index:fold]
    train_fold = np.delete(train_data, np.s_[index:fold],0)

    val_tgt = train_labels[index:fold]
    train_tgt = np.delete(train_labels, np.s_[index:fold])
    
    
    net2.load_weights('net2.h5')
    history = net2.fit(train_fold, train_tgt, epochs = 35, validation_data = (val_fold, val_tgt))

    histories2.append(history)
    
    
    results = net2.evaluate(test_data, test_labels)
    results_list2.append(results)
    print(results)
    
    index = index + val_split_size
    fold = fold + val_split_size

finish = tm.time()
duration2 = finish - start

0
Train on 54 samples, validate on 18 samples
Epoch 1/35
54/54 [==============================] - 2s 29ms/step - loss: 0.8829 - accuracy: 0.7037 - val_loss: 0.7501 - val_accuracy: 0.8333
Epoch 2/35
54/54 [==============================] - 1s 27ms/step - loss: 0.9746 - accuracy: 0.6111 - val_loss: 0.9619 - val_accuracy: 0.5000
Epoch 3/35
54/54 [==============================] - 1s 26ms/step - loss: 0.7657 - accuracy: 0.7778 - val_loss: 0.6463 - val_accuracy: 0.7222
Epoch 4/35
54/54 [==============================] - 1s 26ms/step - loss: 0.7060 - accuracy: 0.8519 - val_loss: 0.5738 - val_accuracy: 0.8333
Epoch 5/35
54/54 [==============================] - 1s 27ms/step - loss: 0.6422 - accuracy: 0.8333 - val_loss: 0.5637 - val_accuracy: 0.8889
Epoch 6/35
54/54 [==============================] - 1s 26ms/step - loss: 0.5553 - accuracy: 0.8704 - val_loss: 0.6151 - val_accuracy: 0.8333
Epoch 7/35
54/54 [==============================] - 1s 26ms/step - loss: 0.4945 - accuracy: 0.8889 - val_los

54/54 [==============================] - 1s 27ms/step - loss: 0.1467 - accuracy: 1.0000 - val_loss: 0.1363 - val_accuracy: 1.0000
Epoch 23/35
54/54 [==============================] - 1s 27ms/step - loss: 0.1381 - accuracy: 1.0000 - val_loss: 0.1239 - val_accuracy: 1.0000
Epoch 24/35
54/54 [==============================] - 1s 27ms/step - loss: 0.1267 - accuracy: 1.0000 - val_loss: 0.1317 - val_accuracy: 1.0000
Epoch 25/35
54/54 [==============================] - 1s 27ms/step - loss: 0.1177 - accuracy: 1.0000 - val_loss: 0.1275 - val_accuracy: 1.0000
Epoch 26/35
54/54 [==============================] - 1s 27ms/step - loss: 0.1090 - accuracy: 1.0000 - val_loss: 0.1250 - val_accuracy: 1.0000
Epoch 27/35
54/54 [==============================] - 1s 27ms/step - loss: 0.1018 - accuracy: 1.0000 - val_loss: 0.1214 - val_accuracy: 1.0000
Epoch 28/35
54/54 [==============================] - 1s 26ms/step - loss: 0.0944 - accuracy: 1.0000 - val_loss: 0.1086 - val_accuracy: 1.0000
Epoch 29/35
54/54 

Epoch 8/35
54/54 [==============================] - 1s 27ms/step - loss: 0.2629 - accuracy: 1.0000 - val_loss: 1.2009 - val_accuracy: 0.5556
Epoch 9/35
54/54 [==============================] - 1s 27ms/step - loss: 0.3570 - accuracy: 0.8704 - val_loss: 1.0052 - val_accuracy: 0.6111
Epoch 10/35
54/54 [==============================] - 1s 26ms/step - loss: 0.3253 - accuracy: 0.9815 - val_loss: 1.1215 - val_accuracy: 0.3889
Epoch 11/35
54/54 [==============================] - 1s 27ms/step - loss: 0.3166 - accuracy: 0.9444 - val_loss: 0.9620 - val_accuracy: 0.6667
Epoch 12/35
54/54 [==============================] - 1s 26ms/step - loss: 0.3005 - accuracy: 0.9074 - val_loss: 0.9932 - val_accuracy: 0.6111
Epoch 13/35
54/54 [==============================] - 1s 27ms/step - loss: 0.2218 - accuracy: 0.9630 - val_loss: 1.3182 - val_accuracy: 0.4444
Epoch 14/35
54/54 [==============================] - 1s 26ms/step - loss: 0.2699 - accuracy: 0.9444 - val_loss: 0.8286 - val_accuracy: 0.6667
Epoch 15

<h2> Expriment3:</h2>

In [16]:
net3 = models.Sequential()
net3.add(layers.Dense((480), input_shape = (480,6)))
net3.add(layers.Flatten())
net3.add(layers.Dense(240,activation='relu'))
net3.add(layers.Dense(240,activation='relu'))
net3.add(layers.Dense(240,activation='tanh'))
net3.add(layers.Dense(7, activation = 'softmax'))
net3.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 480, 480)          3360      
_________________________________________________________________
flatten_4 (Flatten)          (None, 230400)            0         
_________________________________________________________________
dense_17 (Dense)             (None, 240)               55296240  
_________________________________________________________________
dense_18 (Dense)             (None, 240)               57840     
_________________________________________________________________
dense_19 (Dense)             (None, 240)               57840     
_________________________________________________________________
dense_20 (Dense)             (None, 7)                 1687      
Total params: 55,416,967
Trainable params: 55,416,967
Non-trainable params: 0
__________________________________________

In [17]:
net3.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
net3.save_weights('net3.h5')  # each split will start with same weights so can directly compare results

histories3 = []
results_list3 = []
index = 0
fold = val_split_size



start = tm.time()

for i in range (k):
    print(i)
    val_fold = train_data[index:fold]
    train_fold = np.delete(train_data, np.s_[index:fold],0)

    val_tgt = train_labels[index:fold]
    train_tgt = np.delete(train_labels, np.s_[index:fold])
    
    
    net3.load_weights('net3.h5')
    history = net3.fit(train_fold, train_tgt, epochs = 25, validation_data = (val_fold, val_tgt))

    histories3.append(history)
    
    
    results = net3.evaluate(test_data, test_labels)
    results_list3.append(results)
    print(results)
    
    index = index + val_split_size
    fold = fold + val_split_size

finish = tm.time()
duration3 = finish - start

0
Train on 54 samples, validate on 18 samples
Epoch 1/25
54/54 [==============================] - 2s 30ms/step - loss: 3.6235 - accuracy: 0.1481 - val_loss: 2.5521 - val_accuracy: 0.1667
Epoch 2/25
54/54 [==============================] - 1s 26ms/step - loss: 2.7239 - accuracy: 0.1667 - val_loss: 2.2022 - val_accuracy: 0.1667
Epoch 3/25
54/54 [==============================] - 1s 26ms/step - loss: 2.0684 - accuracy: 0.2407 - val_loss: 1.9965 - val_accuracy: 0.1111
Epoch 4/25
54/54 [==============================] - 1s 27ms/step - loss: 1.9246 - accuracy: 0.2593 - val_loss: 1.9404 - val_accuracy: 0.1667
Epoch 5/25
54/54 [==============================] - 1s 27ms/step - loss: 1.8127 - accuracy: 0.3889 - val_loss: 1.7829 - val_accuracy: 0.2778
Epoch 6/25
54/54 [==============================] - 1s 26ms/step - loss: 1.7526 - accuracy: 0.2778 - val_loss: 1.7558 - val_accuracy: 0.2222
Epoch 7/25
54/54 [==============================] - 1s 26ms/step - loss: 1.7257 - accuracy: 0.2222 - val_los

54/54 [==============================] - 1s 26ms/step - loss: 1.5331 - accuracy: 0.4259 - val_loss: 1.7482 - val_accuracy: 0.2222
Epoch 7/25
54/54 [==============================] - 1s 27ms/step - loss: 1.5135 - accuracy: 0.4259 - val_loss: 1.7751 - val_accuracy: 0.2222
Epoch 8/25
54/54 [==============================] - 1s 26ms/step - loss: 1.4862 - accuracy: 0.4815 - val_loss: 1.7825 - val_accuracy: 0.2222
Epoch 9/25
54/54 [==============================] - 1s 27ms/step - loss: 1.4351 - accuracy: 0.3889 - val_loss: 1.6685 - val_accuracy: 0.2778
Epoch 10/25
54/54 [==============================] - 1s 27ms/step - loss: 1.3434 - accuracy: 0.5370 - val_loss: 1.6516 - val_accuracy: 0.2222
Epoch 11/25
54/54 [==============================] - 1s 27ms/step - loss: 1.3431 - accuracy: 0.5185 - val_loss: 1.5828 - val_accuracy: 0.2778
Epoch 12/25
54/54 [==============================] - 1s 27ms/step - loss: 1.2893 - accuracy: 0.6111 - val_loss: 1.5758 - val_accuracy: 0.2778
Epoch 13/25
54/54 [==

In [18]:
avg = np.mean(results_list3, axis=0)
std_dev = np.std(results_list3, axis=0)
print('Average loss and accuracy scores were: ', avg)
print('Standard deviation for loss & accuracy: ', std_dev)
print('Duration in seconds: ', int(duration3))

Average loss and accuracy scores were:  [1.06939955 0.51666668]
Standard deviation for loss & accuracy:  [0.49648185 0.25110866]
Duration in seconds:  145


<h2> Expriment4:</h2>

In [19]:
net4 = models.Sequential()
net4.add(layers.Dense((480), input_shape = (480,6)))
net4.add(layers.Flatten())
net4.add(layers.Dense(240,activation='relu'))
net4.add(layers.Dense(240,activation='relu'))
net4.add(layers.Dense(240,activation='selu'))
net4.add(layers.Dense(7, activation = 'softmax'))
net4.summary()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 480, 480)          3360      
_________________________________________________________________
flatten_5 (Flatten)          (None, 230400)            0         
_________________________________________________________________
dense_22 (Dense)             (None, 240)               55296240  
_________________________________________________________________
dense_23 (Dense)             (None, 240)               57840     
_________________________________________________________________
dense_24 (Dense)             (None, 240)               57840     
_________________________________________________________________
dense_25 (Dense)             (None, 7)                 1687      
Total params: 55,416,967
T

In [20]:
net4.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
net4.save_weights('net4.h5')  # each split will start with same weights so can directly compare results

histories4 = []
results_list4 = []
index = 0
fold = val_split_size



start = tm.time()

for i in range (k):
    print(i)
    val_fold = train_data[index:fold]
    train_fold = np.delete(train_data, np.s_[index:fold],0)

    val_tgt = train_labels[index:fold]
    train_tgt = np.delete(train_labels, np.s_[index:fold])
    
    
    net4.load_weights('net4.h5')
    history = net4.fit(train_fold, train_tgt, epochs = 25, validation_data = (val_fold, val_tgt))

    histories4.append(history)
    
    
    results = net4.evaluate(test_data, test_labels)
    results_list4.append(results)
    print(results)
    
    index = index + val_split_size
    fold = fold + val_split_size

finish = tm.time()
duration4 = finish - start

0
Train on 54 samples, validate on 18 samples
Epoch 1/25
54/54 [==============================] - 2s 30ms/step - loss: 5.3375 - accuracy: 0.1296 - val_loss: 7.7733 - val_accuracy: 0.1667
Epoch 2/25
54/54 [==============================] - 1s 27ms/step - loss: 8.0020 - accuracy: 0.1852 - val_loss: 9.2722 - val_accuracy: 0.2222
Epoch 3/25
54/54 [==============================] - 1s 26ms/step - loss: 8.3820 - accuracy: 0.0926 - val_loss: 5.5761 - val_accuracy: 0.2222
Epoch 4/25
54/54 [==============================] - 1s 27ms/step - loss: 4.0663 - accuracy: 0.2778 - val_loss: 5.1555 - val_accuracy: 0.2778
Epoch 5/25
54/54 [==============================] - 1s 27ms/step - loss: 4.2588 - accuracy: 0.2222 - val_loss: 2.1894 - val_accuracy: 0.1111
Epoch 6/25
54/54 [==============================] - 1s 27ms/step - loss: 2.0328 - accuracy: 0.3148 - val_loss: 2.2488 - val_accuracy: 0.3889
Epoch 7/25
54/54 [==============================] - 1s 27ms/step - loss: 1.7792 - accuracy: 0.4630 - val_los

54/54 [==============================] - 1s 27ms/step - loss: 2.3589 - accuracy: 0.3519 - val_loss: 2.1337 - val_accuracy: 0.3889
Epoch 7/25
54/54 [==============================] - 1s 26ms/step - loss: 2.6098 - accuracy: 0.3333 - val_loss: 1.6161 - val_accuracy: 0.4444
Epoch 8/25
54/54 [==============================] - 1s 27ms/step - loss: 1.6777 - accuracy: 0.5185 - val_loss: 1.4379 - val_accuracy: 0.2778
Epoch 9/25
54/54 [==============================] - 1s 26ms/step - loss: 1.0666 - accuracy: 0.5926 - val_loss: 1.2126 - val_accuracy: 0.3889
Epoch 10/25
54/54 [==============================] - 1s 27ms/step - loss: 1.0745 - accuracy: 0.7222 - val_loss: 0.8527 - val_accuracy: 0.5556
Epoch 11/25
54/54 [==============================] - 1s 27ms/step - loss: 0.8297 - accuracy: 0.7222 - val_loss: 0.7922 - val_accuracy: 0.7778
Epoch 12/25
54/54 [==============================] - 1s 27ms/step - loss: 0.7628 - accuracy: 0.6481 - val_loss: 0.7694 - val_accuracy: 0.6111
Epoch 13/25
54/54 [==

In [21]:
avg = np.mean(results_list4, axis=0)
std_dev = np.std(results_list4, axis=0)
print('Average loss and accuracy scores were: ', avg)
print('Standard deviation for loss & accuracy: ', std_dev)
print('Duration in seconds: ', int(duration4))

Average loss and accuracy scores were:  [0.45946511 0.76666668]
Standard deviation for loss & accuracy:  [0.11426743 0.07453559]
Duration in seconds:  146


<h2> Expriment5:</h2>

In [28]:
net5 = models.Sequential()
net5.add(layers.Dense((480), input_shape = (480,6)))
net5.add(layers.Flatten())
net5.add(layers.Dense(240,activation='selu'))
net5.add(layers.Dense(240,activation='selu'))
net5.add(layers.Dense(240,activation='selu'))
net5.add(layers.Dense(7, activation = 'softmax'))
net5.summary()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_41 (Dense)             (None, 480, 480)          3360      
_________________________________________________________________
flatten_9 (Flatten)          (None, 230400)            0         
_________________________________________________________________
dense_42 (Dense)             (None, 240)               55296240  
_________________________________________________________________
dense_43 (Dense)             (None, 240)               57840     
_________________________________________________________________
dense_44 (Dense)             (None, 240)               57840     
_________________________________________________________________
dense_45 (Dense)             (None, 7)                 1687      
Total params: 55,416,967
T

In [29]:
net5.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
net5.save_weights('net5.h5')  # each split will start with same weights so can directly compare results

histories5 = []
results_list5 = []
index = 0
fold = val_split_size



start = tm.time()

for i in range (k):
    print(i)
    val_fold = train_data[index:fold]
    train_fold = np.delete(train_data, np.s_[index:fold],0)

    val_tgt = train_labels[index:fold]
    train_tgt = np.delete(train_labels, np.s_[index:fold])
    
    
    net5.load_weights('net5.h5')
    history = net5.fit(train_fold, train_tgt, epochs = 25, validation_data = (val_fold, val_tgt))

    histories5.append(history)
    
    
    results = net5.evaluate(test_data, test_labels)
    results_list5.append(results)
    print(results)
    
    index = index + val_split_size
    fold = fold + val_split_size

finish = tm.time()
duration5 = finish - start

0
Train on 54 samples, validate on 18 samples
Epoch 1/25
54/54 [==============================] - 2s 32ms/step - loss: 10.6962 - accuracy: 0.1111 - val_loss: 14.1889 - val_accuracy: 0.0556
Epoch 2/25
54/54 [==============================] - 1s 26ms/step - loss: 17.1072 - accuracy: 0.1667 - val_loss: 21.1174 - val_accuracy: 0.1111
Epoch 3/25
54/54 [==============================] - 1s 27ms/step - loss: 17.3418 - accuracy: 0.1667 - val_loss: 4.3712 - val_accuracy: 0.0556
Epoch 4/25
54/54 [==============================] - 1s 26ms/step - loss: 3.5426 - accuracy: 0.3519 - val_loss: 1.6168 - val_accuracy: 0.3333
Epoch 5/25
54/54 [==============================] - 1s 27ms/step - loss: 3.1897 - accuracy: 0.3889 - val_loss: 2.7532 - val_accuracy: 0.3333
Epoch 6/25
54/54 [==============================] - 1s 27ms/step - loss: 3.3622 - accuracy: 0.5556 - val_loss: 1.9738 - val_accuracy: 0.4444
Epoch 7/25
54/54 [==============================] - 1s 26ms/step - loss: 3.1886 - accuracy: 0.4815 - va

In [30]:
avg = np.mean(results_list5, axis=0)
std_dev = np.std(results_list5, axis=0)
print('Average loss and accuracy scores were: ', avg)
print('Standard deviation for loss & accuracy: ', std_dev)
print('Duration in seconds: ', int(duration5))

Average loss and accuracy scores were:  [0.44806673 0.81666668]
Standard deviation for loss & accuracy:  [0.14883588 0.05527708]
Duration in seconds:  146
